# Experiment 04: Oversampling patches

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E03-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E03-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e04-8', 'e04-6', 'e04-4', 'e04-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[2]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = False         # Step 2 of the training process
PRE_EVALUATE_LAST = True
GAN_EVALUATE_LAST = False
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e04'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 10.0,  ################################# <--- The oversampling of patches
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e04-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e04-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e04-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e04-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e04-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e04-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e04-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e04-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e04-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                         'e04-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                   if k not in ['Coastal', 'NIR2']}, 
                         'e04-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                   if k in ['Blue', 'Green', 'Red', 'NIR']},
                         'e04-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e04-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e04-8': {None: None},                                   # not enough bands in GE01
                         'e04-6': {None: None},                                    # not enough bands in GE01
                         'e04-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                         'e04-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e04-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e04-4': LOGS_EXP_DIR + '/models/' + 'e04-4-pre_20210314-030336/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e04-4': LOGS_EXP_DIR + '/models/' + 'e04-4-gan_20210315-230940/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        

EVAL_STEPS_PER_EPOCH = 'all'
EVAL_N_EPOCHS = 400
#EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
#if PRE_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 1
#    EVAL_PREFIX = EXPERIMENT + '-pre-'
#elif GAN_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 0
#    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: False
NIQE: True
Perceptual Index (PI): False


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 645320, 'val': 8050, 'test': 9315}
Total number of tiles: 662685


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 340.2 and sd 128.5 from json file @ data/toulon-laspezia-tiles/e04/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': 130000,  # 100
                       'val': 8000,  # 100
                       'test': 8000}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.199917335793007
Output (float) min, max: -0.19933143865638941 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH)

Starting matlab.engine ...
matlab.engine started


## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.1 Validation set

In [14]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=False,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_filepath_cache'), 
                                    repeat=False, 
                                    shuffle=False, 
                                    shuffle_buffer_size=0, #SHUFFLE_BUFFER_SIZE[train_val_test], 
                                    include_file_paths=True)
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


#### 8.1.2 Test set

In [15]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [16]:
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    val_or_test = 'val'
    # SENSORS = {'val': ['GE01']}

    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    #if METRIC_MA:
    #    PRE_GAN = ['gan']
    #else:
    #    PRE_GAN = ['pre', 'gan']
    
    if PRE_EVALUATE_LAST and not GAN_EVALUATE_LAST:
        PRE_GAN = ['pre']
    if not PRE_EVALUATE_LAST and GAN_EVALUATE_LAST:
        PRE_GAN = ['gan']

    print(PRE_GAN, SENSORS[val_or_test])
    for pre_gan in PRE_GAN:
        
        for sensor in SENSORS[val_or_test]:
            # if sensor == 'GE01':
            #     continue
                
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            else:
                gan_model.G.load_weights(GAN_WEIGHTS_PATH)

            print(pre_gan, sensor)
            start = time.time()
            results_df = esrgan_evaluate(model=gan_model, 
                                         dataset=ds_val[sensor], 
                                         steps='all', 
                                         per_image=True, 
                                         write_csv=True,
                                         csv_path=str(LOGS_EXP_DIR + '/csv/' + 'final_epoch-' 
                                                      + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                         verbose=1
                                        )
            end = time.time()
            print(str((end - start) / 60), 'minutes')

['pre'] ['WV02', 'GE01']
pre WV02
Computed 8 images in  13.446709632873535 seconds
Last image: {'G_pixel_loss': 5.307237370288931e-05, 'G_perceptual_loss': 0.93897545337677, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 39.94366455078125, 'SSIM': 0.9784864783287048, 'NIQE': 9.934653282165527}
Computed 8 images in  2.527984380722046 seconds
Last image: {'G_pixel_loss': 3.008591193065513e-05, 'G_perceptual_loss': 0.6171953678131104, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 49.26082229614258, 'SSIM': 0.9942283034324646, 'NIQE': 13.734341621398926}
Computed 8 images in  2.4585938453674316 seconds
Last image: {'G_pixel_loss': 0.0001906752586364746, 'G_perceptual_loss': 3.803781032562256, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.10569190979004, 'SSIM': 0.8413370251655579, 'NIQE': 7.62319803237915}
Computed 8 images in  2.440892457962036 seconds
Last image: {'G_pixel_loss': 0.000135601032525300

Computed 8 images in  2.1874349117279053 seconds
Last image: {'G_pixel_loss': 0.00016138743376359344, 'G_perceptual_loss': 3.79738712310791, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.112449645996094, 'SSIM': 0.8447526097297668, 'NIQE': 7.122582912445068}
Computed 8 images in  2.3286306858062744 seconds
Last image: {'G_pixel_loss': 0.0001309852668782696, 'G_perceptual_loss': 2.7966232299804688, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.821407318115234, 'SSIM': 0.8773296475410461, 'NIQE': 7.163211345672607}
Computed 8 images in  2.331019163131714 seconds
Last image: {'G_pixel_loss': 8.186766353901476e-05, 'G_perceptual_loss': 1.0266588926315308, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 39.911842346191406, 'SSIM': 0.9328781366348267, 'NIQE': 8.267863273620605}
Computed 8 images in  2.3385672569274902 seconds
Last image: {'G_pixel_loss': 2.3990078261704184e-05, 'G_perceptual_loss': 0.5

Computed 8 images in  2.2815816402435303 seconds
Last image: {'G_pixel_loss': 8.486762089887634e-05, 'G_perceptual_loss': 1.1455055475234985, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 40.0067138671875, 'SSIM': 0.927729070186615, 'NIQE': 7.19912576675415}
Computed 8 images in  2.362687349319458 seconds
Last image: {'G_pixel_loss': 4.4637916289502755e-05, 'G_perceptual_loss': 0.9085890054702759, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 42.279300689697266, 'SSIM': 0.9869762063026428, 'NIQE': 11.158462524414062}
Computed 8 images in  2.2884416580200195 seconds
Last image: {'G_pixel_loss': 0.0001647807948756963, 'G_perceptual_loss': 3.362332582473755, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.237199783325195, 'SSIM': 0.8595606088638306, 'NIQE': 7.175477027893066}
Computed 8 images in  2.359344244003296 seconds
Last image: {'G_pixel_loss': 6.540731556015089e-05, 'G_perceptual_loss': 0.82783

Computed 8 images in  2.2908787727355957 seconds
Last image: {'G_pixel_loss': 7.478061888832599e-05, 'G_perceptual_loss': 1.0372346639633179, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 40.71357345581055, 'SSIM': 0.9506853222846985, 'NIQE': 8.277782440185547}
Computed 8 images in  2.335846185684204 seconds
Last image: {'G_pixel_loss': 0.00015501082816626877, 'G_perceptual_loss': 3.0386385917663574, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.07596969604492, 'SSIM': 0.7882388234138489, 'NIQE': 6.744349956512451}
Computed 8 images in  2.3411664962768555 seconds
Last image: {'G_pixel_loss': 0.0004784120246767998, 'G_perceptual_loss': 0.8944761753082275, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.901342391967773, 'SSIM': 0.7969887256622314, 'NIQE': 10.916478157043457}
Computed 8 images in  2.2998499870300293 seconds
Last image: {'G_pixel_loss': 0.00012800336116924882, 'G_perceptual_loss': 2.

Computed 8 images in  2.3058559894561768 seconds
Last image: {'G_pixel_loss': 0.00017126512830145657, 'G_perceptual_loss': 3.4032232761383057, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.849044799804688, 'SSIM': 0.7717649340629578, 'NIQE': 6.578266620635986}
Computed 8 images in  2.386937141418457 seconds
Last image: {'G_pixel_loss': 0.00016055266314651817, 'G_perceptual_loss': 2.9850778579711914, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.56092071533203, 'SSIM': 0.7804527282714844, 'NIQE': 6.483767032623291}
Computed 8 images in  2.334322214126587 seconds
Last image: {'G_pixel_loss': 6.540431786561385e-05, 'G_perceptual_loss': 1.1322920322418213, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 41.262332916259766, 'SSIM': 0.9560962915420532, 'NIQE': 7.845973491668701}
Computed 8 images in  2.3352341651916504 seconds
Last image: {'G_pixel_loss': 6.936286808922887e-05, 'G_perceptual_loss': 1.1

...

Computed 8 images in  2.4063422679901123 seconds
Last image: {'G_pixel_loss': 0.0002866180439013988, 'G_perceptual_loss': 5.889604091644287, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.992712020874023, 'SSIM': 0.6763506531715393, 'NIQE': 7.185181617736816}
Computed 8 images in  2.277153491973877 seconds
Last image: {'G_pixel_loss': 5.786971814814024e-05, 'G_perceptual_loss': 1.2094829082489014, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 44.408729553222656, 'SSIM': 0.9813843965530396, 'NIQE': 10.772170066833496}
Computed 8 images in  2.314028739929199 seconds
Last image: {'G_pixel_loss': 0.00022079356131143868, 'G_perceptual_loss': 3.352031946182251, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.27178192138672, 'SSIM': 0.7278156280517578, 'NIQE': 7.6845293045043945}
Computed 8 images in  2.4230141639709473 seconds
Last image: {'G_pixel_loss': 0.00028173919417895377, 'G_perceptual_loss': 5.1

Computed 8 images in  2.335468292236328 seconds
Last image: {'G_pixel_loss': 0.000256467261351645, 'G_perceptual_loss': 5.830990791320801, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.915897369384766, 'SSIM': 0.7836049795150757, 'NIQE': 7.436713695526123}
Computed 8 images in  2.381603479385376 seconds
Last image: {'G_pixel_loss': 0.00025581903173588216, 'G_perceptual_loss': 5.888580322265625, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.756168365478516, 'SSIM': 0.7721121311187744, 'NIQE': 7.399893760681152}
Computed 8 images in  2.454782009124756 seconds
Last image: {'G_pixel_loss': 0.00023227579367812723, 'G_perceptual_loss': 5.075435161590576, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.68811798095703, 'SSIM': 0.8352566361427307, 'NIQE': 6.91515588760376}
Computed 8 images in  2.4237236976623535 seconds
Last image: {'G_pixel_loss': 0.000210551981581375, 'G_perceptual_loss': 2.75142025

Computed 8 images in  2.4065427780151367 seconds
Last image: {'G_pixel_loss': 0.0001682173606241122, 'G_perceptual_loss': 0.6903106570243835, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 37.0302734375, 'SSIM': 0.9879679679870605, 'NIQE': 15.664999961853027}
Computed 8 images in  2.358602285385132 seconds
Last image: {'G_pixel_loss': 0.00024340230447705835, 'G_perceptual_loss': 3.528502941131592, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.7157039642334, 'SSIM': 0.841789722442627, 'NIQE': 7.393754959106445}
Computed 8 images in  2.3485140800476074 seconds
Last image: {'G_pixel_loss': 0.00020822502847295254, 'G_perceptual_loss': 1.0480456352233887, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.71578598022461, 'SSIM': 0.9330161809921265, 'NIQE': 9.888204574584961}
Computed 8 images in  2.3547286987304688 seconds
Last image: {'G_pixel_loss': 0.0003196887846570462, 'G_perceptual_loss': 6.35650157

Computed 8 images in  2.4250802993774414 seconds
Last image: {'G_pixel_loss': 0.0005203222390264273, 'G_perceptual_loss': 2.885784149169922, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.060659408569336, 'SSIM': 0.8322967886924744, 'NIQE': 13.218732833862305}
Computed 8 images in  2.3476309776306152 seconds
Last image: {'G_pixel_loss': 0.0009282620158046484, 'G_perceptual_loss': 5.441664218902588, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 20.202056884765625, 'SSIM': 0.23650741577148438, 'NIQE': 6.682929992675781}
Computed 8 images in  2.3643059730529785 seconds
Last image: {'G_pixel_loss': 0.0004816066357307136, 'G_perceptual_loss': 0.8657478094100952, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.903841018676758, 'SSIM': 0.8637521862983704, 'NIQE': 18.60725975036621}
Computed 8 images in  2.320219039916992 seconds
Last image: {'G_pixel_loss': 0.0006324817077256739, 'G_perceptual_loss': 2.2

Computed 8 images in  2.4596076011657715 seconds
Last image: {'G_pixel_loss': 0.0005534264491870999, 'G_perceptual_loss': 2.7701148986816406, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 25.31599235534668, 'SSIM': 0.8070273399353027, 'NIQE': 13.36894702911377}
Computed 8 images in  2.359259843826294 seconds
Last image: {'G_pixel_loss': 0.0004495455650612712, 'G_perceptual_loss': 0.7745421528816223, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.489463806152344, 'SSIM': 0.8898085355758667, 'NIQE': 18.343313217163086}
Computed 8 images in  2.3101980686187744 seconds
Last image: {'G_pixel_loss': 0.0006737100193277001, 'G_perceptual_loss': 3.824087142944336, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 22.999879837036133, 'SSIM': 0.7471191883087158, 'NIQE': 8.12618350982666}
Computed 8 images in  2.4105567932128906 seconds
Last image: {'G_pixel_loss': 0.00117135897744447, 'G_perceptual_loss': 6.45573

### 8.3 Evaluate every kth epoch

In [17]:
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    val_or_test = 'val'
    
    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    if METRIC_MA:
        PRE_GAN = ['gan']
    else:
        PRE_GAN = ['pre', 'gan']

    for pre_gan in PRE_GAN:
        for sensor in SENSORS[val_or_test]:
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                model_weights_dir = PRETRAIN_WEIGHTS_DIR
                eval_first_step = 1
                eval_prefix = EXPERIMENT + '-pre-'
            else:
                model_weights_dir = GAN_WEIGHTS_DIR
                eval_first_step = 0
                eval_prefix = EXPERIMENT + '-gan-'

            esrgan_epoch_evaluator(gan_model,
                                   model_weights_dir=model_weights_dir,
                                   model_weight_prefix=eval_prefix,
                                   dataset=ds_val[sensor],
                                   n_epochs=EVAL_N_EPOCHS,
                                   first_epoch=eval_first_step,
                                   steps_per_epoch=EVAL_STEPS_PER_EPOCH,
                                   k_epoch=25,
                                   csv_dir=LOGS_EXP_DIR + '/csv/' + val_or_test + '-' + sensor, 
                                   per_image=EVAL_PER_IMAGE, 
                                   verbose=0)

### 8.4 Comparison plots